![SAP Ariba logo](./images/sap-ariba-small.jpeg)  

# Master Data Retrieval API for Procurement

## Overview

SAP Ariba solutions store master data loaded from external systems in the master data service. This feature introduces the Master Data Retrieval API to retrieve the stored master data. Customers can create a client application on the SAP Ariba developer portal to filter the master data and download it.

Master data loaded from external systems is stored in the master data service. This feature introduces the Master Data Retrieval API to retrieve the stored master data. Customers can create a client application on the SAP Ariba developer portal operated by CDC to filter the master data and download it.

**API	Documentation**	
https://help.sap.com/viewer/b62431d654644850ab1ca6ba6a4c532e/cloud/en-US

##  Initialize scripts

In [44]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
from IPython.display import display

In [45]:
def get_access_token(base64_auth):
    payload = {}
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Basic {base64_auth}"
    }
    
    response = requests.request("POST", OAUTH_API_URL, headers=headers, data=payload)
    
    return response.json()["access_token"]

## Setup API details

In [46]:
# authentication server
OAUTH_API_URL = "https://api.ariba.com/v2/oauth/token?grant_type=client_credentials"


REALM = "YOUR REALM" # add your realm name

# API credentials and endpoints

MASTER_DATA_BASEURL = "	https://openapi.ariba.com/api/mds-search/v1/prod"


MASTER_DATA_APIKEY = "YOUR API KEY"# add your API key
MASTER_DATA_BASE64 = "YOUR BASE64 SECRET" # add your BASE64 secret


In [47]:
MASTER_DATA_TOKEN = get_access_token(MASTER_DATA_BASE64)
MASTER_DATA_TOKEN

'f3206707-57fd-45c1-bd33-8d7b7b5e7e9e'

## Set headers

In [48]:
api_headers = {
        "Authorization": f"Bearer {MASTER_DATA_TOKEN}",
        "X-Realm": REALM,
        "apiKey": MASTER_DATA_APIKEY,
        "Accept-Language": "en",
    }

#



# retrieve all entitiy types available

api_response = requests.get(
    MASTER_DATA_BASEURL + "/entityTypes",
    headers=api_headers,
)



## Retrieve /entityTypes endpoint

In [49]:
# retrieve all entitiy types available

api_response = requests.get(
    MASTER_DATA_BASEURL + "/entityTypes",
    headers=api_headers,
)


data = json.loads(api_response.text)

#convert to DataFrame
entities = pd.DataFrame(data)

#displaying invoices as a nice table html
display(entities)

,EntityName,Name
0,groups,Group
1,uoms,UnitOfMeasure
2,users,User
3,currency,Currency
4,currencyconversionrate,CurrencyConversionRate
5,localeids,LocaleID
6,countries,Country
7,classificationcodemap,ClassificationCodeMap
8,commoditycodes,CommodityCode
9,languages,Language


## Retrieve /entityTypes/{EntityName} endpoint

In [50]:
# set EntityName you want to get metadata from (see table above)
entity_name = "suppliers" # let's use suppliers for this example

# retrieve the metadata from specific MD

api_response = requests.get(
    MASTER_DATA_BASEURL + "/entityTypes/" + entity_name,
    headers=api_headers,
)

data = json.loads(api_response.text)

data


{'properties': {'IsOneTimeVendorShell': {'isNullable': True,
   'type': 'boolean'},
  'CommonSupplier': {'isRef': True,
   'isNullable': True,
   'type': 'object',
   'externalRef': 'https://openapi.ariba.com/api/mds-search/v1/prod/entityTypes/commonsuppliers'},
  'ProcurementUnits': {'isVariantField': True,
   'isRef': True,
   'isNullable': True,
   'isVector': True,
   'type': 'array',
   'items': {'type': 'object',
    'externalRef': 'https://openapi.ariba.com/api/mds-search/v1/prod/entityTypes/procurementunits'}},
  'CarrierMethod': {'isNullable': True, 'TextLength': '200', 'type': 'string'},
  'Locations': {'isRef': True,
   'isNullable': True,
   'isVector': True,
   'type': 'array',
   'items': {'type': 'object',
    'externalRef': 'https://openapi.ariba.com/api/mds-search/v1/prod/entityTypes/supplierloc'}},
  'Text': {'isNullable': True, 'TextLength': '255', 'type': 'string'},
  'PaymentTerms': {'isRef': True,
   'isNullable': True,
   'type': 'object',
   'externalRef': 'http

## Retrieve /entities/{EntityName} endpoint


In [51]:
# set EntityName you want to get metadata from 
entity_name = "purchseorgs" # let's use purchase organizations for this example


# set filters as necessary - check documentation for all available filters
api_params = {"$top": "1",
             }


# retrieve the metadata from specific MD
api_response = requests.get(
    MASTER_DATA_BASEURL + "/entities/" + entity_name,
    headers=api_headers,
    params=api_params,
)

data = json.loads(api_response.text)

#flattening json
list_of_porgs= pd.json_normalize(data, max_level=1)

#transforming into DataFrame
list_of_porgs  = pd.DataFrame(list_of_porgs)

#displaying invoices as a nice table html
display(list_of_porgs)



,TimeCreated,Active,SystemId,TimeUpdated,UniqueName,Name,CompanyCode.UniqueName
0,2019-03-19 12:20:20.379,true,ED7CLNT811-AN01426057983-T,2020-12-04 12:28:14.089,3000,IDES USA,3000


## Retrieve /entities/{EntityName}/count endpoint


In [52]:
# set EntityName you want to get metadata from 
entity_name = "users" # let's use users for this example

# retrieve the metadata from specific MD
api_response = requests.get(
    MASTER_DATA_BASEURL + "/entities/" + entity_name + "/$count",
    headers=api_headers,
    params=api_params,
)


data = json.loads(api_response.text)
print("Total number of users: ", data)



Total number of users:  239
